In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import csv
import time
from collections import defaultdict, Counter
import json
from urllib.request import Request, urlopen
import pickle

In [6]:
import pandas as pd
import numpy as np
import geopandas as gpd
import shapely
from shapely.geometry import shape, Point
import shapefile
from geopy import distance
import plotly.figure_factory as ff
import plotly.io as pio
import plotly as plt
pio.renderers.default = "jupyterlab"
import plotly.express as px
import plotly.graph_objects as go

In [7]:
from tqdm import tqdm

In [8]:
from utils import *

In [9]:
df = load_csv_export_dataset("export_dataframe_stage2.csv")

/Users/robclewley/.pyenv/versions/3.7.5/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning:

Columns (14,20,21,23,24,31,32,33,35,36,56,63,64,65,72,73,74,77,78,79,86,87,88) have mixed types.Specify dtype option on import or set low_memory=False.



In [35]:
loc_pairs, geo_locs, gdf = get_loc_objects_from_series(locs['SiteLocation'])

# Distribution of locations

Make sure to unzip the file in the geodata folder first.

In [218]:
# watershed boundaries
WBD_gj = shapefile.Reader("geodata/hydrologic_units_WBDHU12_ga_3975106_02/hydrologic_units\wbdhu12_a_ga.shp").__geo_interface__

Demonstrate that the max inter-site distance is < 450 miles

In [84]:
GA_map = get_state_map()

In [21]:
GA_map.head()

,FIPS,STNAME,County,TOT_POP,TOT_MALE,TOT_FEMALE,WA_MALE,WA_FEMALE,NHWA_MALE,NHWA_FEMALE,NHWhite_Alone,Not_NHWhite_Alone,MinorityMinority,MinorityPCT,Black,BlackPCT,Hispanic,HispanicPCT
2807,13001,Georgia,Appling,18368,9294,9074,7197,7139,6284,6520,12804,5564,No,30.29%,3607,20%,1728,9.41%
2808,13003,Georgia,Atkinson,8284,4203,4081,3291,3143,2317,2371,4688,3596,No,43.41%,1459,18%,2050,24.75%
2809,13005,Georgia,Bacon,11198,5562,5636,4558,4592,4094,4273,8367,2831,No,25.28%,1790,16%,890,7.95%
2810,13007,Georgia,Baker,3366,1606,1760,858,876,796,823,1619,1747,Yes,51.90%,1551,46%,152,4.52%
2811,13009,Georgia,Baldwin,46367,23365,23002,13011,12750,12638,12391,25029,21338,No,46.02%,19150,41%,977,2.11%


In [85]:
fips = GA_map['FIPS'].tolist()

In [229]:
wbd_names = set([wbd['properties']['name'] for wbd in WBD_gj['features']])

### Plot distribution of unique record locations using mapbox

In [ ]:
fig = px.choropleth_mapbox(pd.DataFrame(data={'name': [wbd['properties']['name'] for wbd in WBD_gj['features']]}),
                           geojson=WBD_gj,
                           locations="name",
                           featureidkey="properties.name",
                           center={"lat": 32.8407, "lon": -83.6324}, # macon
                           mapbox_style="stamen-terrain", #"carto-positron",
                           zoom=6,
                           opacity=0.3,
                      )

fig.add_scattermapbox(
    lat = gdf.geometry.x,
    lon = gdf.geometry.y,
    mode = 'markers',
    hovertext = [d['County']['name'] for d in loc_lookup],
    marker_size=4,
    marker_color='rgb(235, 0, 100)'
)

fig.update_layout(margin={"r":0, "t":0, "l":0, "b":0})
fig.show()

## Lookup GIS place names associated with the recordings

In [40]:
loc_lookup = fetch_geo_locs()

In [41]:
loc_lookup[3]['State']['name']

'Florida'

In [42]:
loc_lookup[155]['State']['name']

'Georgia'

Example location info

In [43]:
loc_lookup[0]

{'Block': {'FIPS': None, 'bbox': None},
 'County': {'FIPS': None, 'name': None},
 'State': {'FIPS': None, 'code': None, 'name': None},
 'status': 'OK',
 'executionTime': '0'}

In [44]:
loc_lookup[-1]

{'messages': ["FCC0001: The coordinate lies on the boundary of mulitple blocks, the block contains the clicked location is selected. For a complete list use showall=true to display 'intersection' element in the Block"],
 'Block': {'FIPS': '370399301002000',
  'bbox': [-83.7769, 35.212586, -83.702902, 35.25766]},
 'County': {'FIPS': '37039', 'name': 'Cherokee'},
 'State': {'FIPS': '37', 'code': 'NC', 'name': 'North Carolina'},
 'status': 'OK',
 'executionTime': '0'}

In [267]:
distance.distance(loc_pairs[3], loc_pairs[155]).miles

426.12898114111164

In [476]:
df[['lat', 'long', 'SiteLocation', 'SiteName', 'wbd']].tail()

,lat,long,SiteLocation,SiteName,wbd
63792,32.0067,-84.2266,32.0067 -84.2266,Muckalee Creek at McLittle Bridge Rd,Bear Branch
63793,31.2387,-82.3231,31.2387 -82.3231,"U.S. Highway 84, Waycross",Caney Branch-Satilla River
63794,31.2933,-81.9571,31.2933 -81.9571,"U.S. 301, Nahunta",Raulerson Swamp-Satilla River
63795,31.2196,-81.8677,31.2196 -81.8677,"U.S. Highway 82, Nahunta",Barbers Creek-Satilla River
63796,31.2400,-81.8634,31.24 -81.8634,Warners Landing,Barbers Creek-Satilla River


We'll keep "SiteLocation" to allow easy checking of unique lat-long pairs

# Coverage of WBDs by records

In [456]:
wbd_counts = df.groupby('wbd').agg(count=('wbd', 'count'))

In [452]:
wbd_no_records = set(wbd_names).difference(df['wbd'].unique())

In [458]:
wbd_counts.sort_values(by='count').head(10)

,count
wbd,
Shaddock Creek-Oconee River,1
Upper Tesnatee Creek,1
Heiferhorn Creek,1
Town of Pelham-Big Slough,1
Pigeon Creek,1
Harris Creek-St. Marys River,1
Hardy Mill Creek,1
Upper Lanahassee Creek,1
Gola Creek-Potato Creek,1


In [460]:
poorly_repd_wbds = wbd_counts[wbd_counts['count'] < 50]
well_repd_wbds = wbd_counts[wbd_counts['count'] >= 50]

In [463]:
len(wbd_no_records), len(poorly_repd_wbds), len(well_repd_wbds)

(1093, 376, 188)

In [464]:
completion_by_fieldgroup(df[df['wbd'].isin(set(well_repd_wbds.index))])

{'core': 0.925784996133024,
 'basic': 0.30055928684821104,
 'chem': 0.1986981649441046,
 'bact': 0.20278669760247486,
 'other': 0.0006032482598607888}

In [465]:
completion_by_fieldgroup(df[df['wbd'].isin(set(poorly_repd_wbds.index))])

{'core': 0.9675754910583406,
 'basic': 0.36397722538536315,
 'chem': 0.24142480211081793,
 'bact': 0.1798182351216652,
 'other': 0.0007538635506973238}

# Coverage of counties by records

In [466]:
county_counts = df.groupby('county').agg(count=('county', 'count'))

In [468]:
county_no_records = set(counties).difference(df['county'].unique())

In [470]:
county_counts.sort_values(by='count').head(10)

,count
county,
Jones,1
Anderson,1
Marion,1
Wayne,1
Laurens,1
Hancock,1
Chattahoochee,1
Irwin,2
Webster,2


In [471]:
poorly_repd_counties = county_counts[county_counts['count'] < 50]
well_repd_counties = county_counts[county_counts['count'] >= 50]

In [472]:
len(county_no_records), len(poorly_repd_counties), len(well_repd_counties)

(14, 45, 90)

In [473]:
completion_by_fieldgroup(df[df['county'].isin(set(well_repd_counties.index))])

{'core': 0.9294503389330976,
 'basic': 0.3060116624963756,
 'chem': 0.20251973932345285,
 'bact': 0.20020063932530777,
 'other': 0.0005963311666229438}

In [474]:
completion_by_fieldgroup(df[df['county'].isin(set(poorly_repd_counties.index))])

{'core': 0.9573791348600509,
 'basic': 0.3544931029864738,
 'chem': 0.22374508443210733,
 'bact': 0.22357930449533503,
 'other': 0.0021810250817884407}

# Q. Is it always the same groups that measure at the same locations?

In [400]:
df['group_rid'].nunique()

1112

In [401]:
df.groupby('group_rid').count()

,GroupName,site_rid,state,county,lat,long,SiteName,SiteLocation,event_rid,event_date,...,hold_end_datetime,min_temp,max_temp,three_M_plate,ecoli_idexx,fecal_coliform,ecoli_other,ecoli_other_unit,comments,warnings
group_rid,,,,,,,,,,,,,,,,,,,,,
104,3,3,3,3,3,3,3,3,3,3,...,0,0,0,0,0,0,0,0,0,0
130,40,40,40,40,40,40,40,40,40,40,...,0,0,0,0,0,0,0,0,0,0
133,13,13,13,13,13,13,13,13,13,13,...,0,0,0,0,0,0,0,0,0,0
134,3,3,3,3,3,3,3,3,3,3,...,0,0,0,0,0,0,0,0,0,0
137,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2997,3,3,3,3,3,3,3,3,3,3,...,0,0,0,0,0,0,0,0,0,0
3006,2,2,2,2,2,2,2,2,2,2,...,0,0,0,0,0,0,0,0,0,0
3007,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [403]:
df[['group_rid','SiteName']].groupby('SiteName').count()

,group_rid
SiteName,
Ivy Creek & Little Ivy Creek,26
#1b Disharoon Creek before it enters Disharoon Lake,3
#22 Key Largo,8
#5a Disharoon Creek,49
#6 East Branch Long Swamp Creek,7
...,...
up stream Trib of Peachtree Creek,1
upper cherry creek,8
white marlin nautilus dr,1


In [404]:
df[['group_rid','SiteLocation']].groupby('SiteLocation').count()

,group_rid
SiteLocation,
18.97 -96.0333,5
24.2808 -82.2217,4
24.5251 -81.5192,3
24.5457 -81.7836,1
24.5463 -81.7945,2
...,...
35.2064 -83.8156,36
35.2112 -83.3252,2
35.2297 -83.7392,3


<br><hr><br>